In [1]:
from saflow.data import load_fooof_data, balance_data, balance_dataset, select_trial, select_epoch, get_VTC_bounds, get_inout
from saflow.utils import create_fnames
import os.path as op
import pickle
import saflow
import numpy as np
import os


In [2]:
subject = '04'
run = '02'

fnames = create_fnames(subject, run)
psd_fname = str(fnames['welch'].fpath) + '.pkl'
with open(psd_fname, 'rb') as f:
    psds = pickle.load(f)
    events_dicts = psds['info']

In [21]:
## Selecting epochs
inbound, outbound = get_VTC_bounds(events_dicts, lowbound=25, highbound=75)
inout_epoch = get_inout(events_dicts[0], inbound, outbound)
epoch_selected = select_epoch(events_dicts[0], bad_how='any', type_how='alltrials', inout_epoch=inout_epoch, verbose=True)


Bad : True, InOut : True, Type : True, Retain : False
['correct_commission' 'correct_commission' 'correct_commission'
 'correct_commission' 'correct_commission' 'correct_commission'
 'correct_commission' 'correct_commission']
[False False False False False  True  True  True]


In [4]:
feature_fpath = op.join(saflow.BIDS_PATH, 'derivatives', 'fooof_fixed_alltrials_1022_sensor_8trials')

X = []
y = []
groups = []
VTC = []
task = []
for sub in sorted(os.listdir(feature_fpath)):
    for file in sorted(os.listdir(op.join(feature_fpath, sub, 'meg'))):
        print(file)
        if 'magic' in file:
            fname = op.join(feature_fpath, sub, 'meg', file)
            fname_output = fname.replace('.pkl', '_magic.pkl')
            with open(fname, 'rb') as f:
                data = pickle.load(f)
            # Reshape data
            n_chans = 270
            n_trials = int(len(data)/n_chans)
            data_reshaped = np.array(data).reshape(n_trials, n_chans)
            # Get events dicts
            events_dicts = get_events_dicts_from_magic_dict(data_reshaped)
            inbound, outbound = get_VTC_bounds(events_dicts, lowbound=25, highbound=75)

sub-04_task-gradCPT_meg.pkl
sub-04_task-gradCPT_run-02_meg.pkl
sub-04_task-gradCPT_run-02_meg_magic.pkl
sub-04_task-gradCPT_run-03_meg.pkl
sub-04_task-gradCPT_run-03_meg_magic.pkl
sub-04_task-gradCPT_run-04_meg.pkl
sub-04_task-gradCPT_run-04_meg_magic.pkl
sub-04_task-gradCPT_run-05_meg.pkl
sub-04_task-gradCPT_run-05_meg_magic.pkl
sub-04_task-gradCPT_run-06_meg.pkl
sub-04_task-gradCPT_run-06_meg_magic.pkl
sub-04_task-gradCPT_run-07_meg.pkl
sub-04_task-gradCPT_run-07_meg_magic.pkl
sub-05_task-gradCPT_meg.pkl
sub-05_task-gradCPT_run-02_meg.pkl
sub-05_task-gradCPT_run-02_meg_magic.pkl


KeyboardInterrupt: 

In [10]:
def get_events_dicts_from_magic_dict(data_reshaped):
    events_dicts = []
    for trial_idx in range(data_reshaped.shape[0]):
        events_dicts.append(data_reshaped[trial_idx][0]['info'])
    return events_dicts

events_dicts = get_events_dicts_from_magic_dict(data_reshaped)

[{'event_idx': 17,
  't0_sample': 7684,
  'VTC': 0.5630313654528545,
  'task': 'correct_commission',
  'RT': 0.5524047746075667,
  'INOUT': 'OUT',
  'INOUT_2575': nan,
  'INOUT_1090': nan,
  'bad_epoch': False,
  'included_bad_epochs': array([False, False, False, False, False, False, False, False]),
  'included_events_idx': [2, 5, 7, 9, 11, 13, 15, 17],
  'included_VTC': array([0.72316776, 0.52793364, 1.52745696, 1.8022818 , 0.85820685,
         0.87548742, 0.884918  , 0.56303137]),
  'included_task': array(['correct_commission', 'correct_commission', 'correct_commission',
         'correct_commission', 'correct_commission', 'correct_commission',
         'correct_commission', 'correct_commission'], dtype=object),
  'included_RT': array([0.75305354, 0.55788007, 0.40195311, 0.35908007, 0.50635701,
         0.50366122, 0.50219003, 0.55240477]),
  'included_INOUT': array(['IN', 'IN', 'IN', 'IN', 'OUT', 'OUT', 'OUT', 'OUT'], dtype=object),
  'included_INOUT_2575': array(['IN', nan, nan, na